In [1]:
# WGAN-GP for CelebA 64x64 Face Generation
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import os

# -----------------------
# 1. Models: Generator & Critic
# -----------------------
class Generator(nn.Module):
    def __init__(self, z_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 4, 1, 0),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

class Critic(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.InstanceNorm2d(128, affine=True),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.InstanceNorm2d(256, affine=True),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, 4, 2, 1),
            nn.InstanceNorm2d(512, affine=True),
            nn.LeakyReLU(0.2),
            nn.Conv2d(512, 1, 4, 1, 0)
        )

    def forward(self, x):
        return self.net(x).view(-1)

# -----------------------
# 2. Gradient Penalty
# -----------------------
def gradient_penalty(critic, real, fake, device="cuda"):
    batch_size, c, h, w = real.shape
    epsilon = torch.rand((batch_size, 1, 1, 1), device=device).repeat(1, c, h, w)
    interpolated = real * epsilon + fake * (1 - epsilon)
    interpolated.requires_grad_(True)

    mixed_scores = critic(interpolated)
    gradient = torch.autograd.grad(
        inputs=interpolated,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True
    )[0]

    gradient = gradient.view(gradient.shape[0], -1)
    gp = ((gradient.norm(2, dim=1) - 1)**2).mean()
    return gp

# -----------------------
# 3. Training Setup
# -----------------------
image_dir = "celebA/celeba/img_align_celeba"
os.makedirs("wgan_outputs", exist_ok=True)

transform = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder(root=image_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

z_dim = 128
gen = Generator(z_dim).to(device)
critic = Critic().to(device)
opt_gen = torch.optim.Adam(gen.parameters(), lr=1e-4, betas=(0.0, 0.9))
opt_critic = torch.optim.Adam(critic.parameters(), lr=1e-4, betas=(0.0, 0.9))

# -----------------------
# 4. Training Loop
# -----------------------
critic_iters = 5
epochs = 3
lambda_gp = 10

for epoch in range(epochs):
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        cur_batch_size = real.size(0)

        # Train critic
        for _ in range(critic_iters):
            z = torch.randn(cur_batch_size, z_dim, 1, 1).to(device)
            fake = gen(z).detach()
            critic_real = critic(real)
            critic_fake = critic(fake)
            gp = gradient_penalty(critic, real, fake, device=device)
            loss_critic = -(torch.mean(critic_real) - torch.mean(critic_fake)) + lambda_gp * gp

            opt_critic.zero_grad()
            loss_critic.backward()
            opt_critic.step()

        # Train generator
        z = torch.randn(cur_batch_size, z_dim, 1, 1).to(device)
        fake = gen(z)
        loss_gen = -torch.mean(critic(fake))

        opt_gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] Batch {batch_idx}/{len(dataloader)} "
                  f"Loss D: {loss_critic.item():.4f}, loss G: {loss_gen.item():.4f}")

    # Save samples
    with torch.no_grad():
        z = torch.randn(64, z_dim, 1, 1).to(device)
        samples = gen(z)
        samples = (samples + 1) / 2
        save_image(samples, f"wgan_outputs/epoch_{epoch+1}.png", nrow=8)

# -----------------------
# 5. Generate 100,000 Images
# -----------------------
print("Generating 100,000 images...")
output_dir = "wgan_outputs/generated"
os.makedirs(output_dir, exist_ok=True)

gen.eval()
batch_size = 100
for i in range(0, 100000, batch_size):
    z = torch.randn(batch_size, z_dim, 1, 1).to(device)
    with torch.no_grad():
        fake = gen(z).cpu()
        fake = (fake + 1) / 2
        for j in range(fake.size(0)):
            idx = i + j
            save_image(fake[j], f"wgan_generated/generated/image_{idx:05d}.png")

print("Done. 100,000 images saved to 'wgan_generated/'")

/sciclone/data10/iahewababarand/genVision-VAE/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


/sciclone/data10/iahewababarand/genVision-VAE/.venv/lib64/python3.9/site-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/sciclone/data10/iahewababarand/genVision-VAE/.venv/lib64/python3.9/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch [1/3] Batch 0/3166 Loss D: -6.3916, loss G: 4.5593


Epoch [1/3] Batch 100/3166 Loss D: -26.6402, loss G: 21.0721


Epoch [1/3] Batch 200/3166 Loss D: -19.2689, loss G: 33.9940


...

Epoch [1/3] Batch 3100/3166 Loss D: -12.3521, loss G: 59.8272


Epoch [2/3] Batch 0/3166 Loss D: -11.0298, loss G: 55.2957


Epoch [2/3] Batch 100/3166 Loss D: -14.7308, loss G: 63.5315


Epoch [2/3] Batch 200/3166 Loss D: -14.1501, loss G: 57.8809


Epoch [2/3] Batch 300/3166 Loss D: -14.6483, loss G: 48.4475
